<a href="https://colab.research.google.com/github/Vikas-KM/machine-learning/blob/master/ML_Projects/Movie%20Recommendation%20System/Movie_Recommendation_using_Naive_Bayes_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movie Recommendation System using Naive Bayes 

Articles Referred
- https://iopscience.iop.org/article/10.1088/1742-6596/1345/4/042042/pdf
  -  Liu Shuxian and Fang Sen 2019 J. Phys.: Conf. Ser. 1345 042042

Comparative Analysis of different ML Algorithms for recommendation systems
- https://ijcat.com/archives/volume6/issue2/ijcatr06021005.pdf

Movielens Dataset Referred
- https://grouplens.org/datasets/movielens/

link - http://files.grouplens.org/datasets/movielens/ml-latest-small-README.html

### About the dataset
- This **dataset (ml-25m)** describes 5-star rating and free-text tagging activity from MovieLens, a movie recommendation service. **It contains 25000095 ratings and 1093360 tag applications across 62423 movies.** These data were created by 162541 users between January 09, 1995 and November 21, 2019. **This dataset was generated on November 21, 2019.**
- Only movies with at least one rating or tag are included in the dataset. These movie ids are consistent with those used on the MovieLens web site (e.g., id 1 corresponds to the URL https://movielens.org/movies/1). Movie ids are consistent between ratings.csv, tags.csv, movies.csv, and links.csv (i.e., the same id refers to the same movie across these four data files).

### Ratings Data File Structure (ratings.csv)
- All ratings are contained in the file ratings.csv. Each line of this file after the header row represents one rating of one movie by one user, and has the following format:

- userId,movieId,rating,timestamp
The lines within this file are ordered first by userId, then, within user, by movieId.

- Ratings are made on a 5-star scale, with half-star increments (0.5 stars - 5.0 stars).

- Timestamps represent seconds since midnight Coordinated Universal Time (UTC) of January 1, 1970.


### Tags Data File Structure (tags.csv)
- All tags are contained in the file tags.csv. Each line of this file after the header row represents one tag applied to one movie by one user, and has the following format:

- userId,movieId,tag,timestamp
- The lines within this file are ordered first by userId, then, within user, by movieId.

- Tags are user-generated metadata about movies. Each tag is typically a single word or short phrase. The meaning, value, and purpose of a particular tag is determined by each user.

- Timestamps represent seconds since midnight Coordinated Universal Time (UTC) of January 1, 1970.

### Movies Data File Structure (movies.csv)
- Movie information is contained in the file movies.csv. Each line of this file after the header row represents one movie, and has the following format:

- movieId,title,genres
- Movie titles are entered manually or imported from https://www.themoviedb.org/, and include the year of release in parentheses. Errors and inconsistencies may exist in these titles.

- Genres are a pipe-separated list, and are selected from the following:

  - Action
  - Adventure
  - Animation
  - Children's
  - Comedy
  - Crime
  - Documentary
  - Drama
  - Fantasy
  - Film-Noir
  - Horror
  - Musical
  - Mystery
  - Romance
  - Sci-Fi
  - Thriller
  - War
  - Western
  - (no genres listed)

In [1]:
# importing some standard libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
%matplotlib inline

In [3]:
# Mounting the google drive to download the dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Lets Import the movielens dataset
# using movielens 25M dataset

! ls /content/drive/MyDrive/ML_Datasets/movielens_dataset

genome-scores.csv  links.csv   ratings.csv  tags.csv
genome-tags.csv    movies.csv  README.txt


### movie.csv that contains movie information:

In [118]:
df_movies = pd.read_csv('/content/drive/MyDrive/ML_Datasets/movielens_dataset/movies.csv')
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [119]:
# shape of the movies dataset
df_movies.shape

(62423, 3)

In [120]:
# columns in the movies dataset
df_movies.columns

Index(['movieId', 'title', 'genres'], dtype='object')

In [121]:
# lets check if there any null values
df_movies.isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

In [122]:
# genre needs to be seprated, before we can figure which genre is dominating
df_movies['genres'].value_counts()

Drama                                               9056
Comedy                                              5674
(no genres listed)                                  5062
Documentary                                         4731
Comedy|Drama                                        2386
                                                    ... 
Animation|Children|Comedy|Musical|Romance              1
Action|Children|Drama|Thriller                         1
Action|Adventure|Comedy|Musical|Romance|Thriller       1
Children|Fantasy|Thriller                              1
Comedy|Drama|Horror|Romance|Thriller                   1
Name: genres, Length: 1639, dtype: int64

### Observations from the movies.csv dataset
- Title contains the year, needs to be cleaned
- Single movie can have many genres, needs to split
- WOW - No Null data

In [123]:
print(len(df_movies['movieId'].unique()))
print(len(df_movies['title'].unique()))

62423
62325


In [124]:
# list of movies which are duplicate
df_movies_dup = df_movies[df_movies.duplicated(subset=['title'], keep=False)]
df_movies_dup.head()

,movieId,title,genres
580,588,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical
1710,1788,Men with Guns (1997),Action|Drama
2553,2644,Dracula (1931),Horror
2759,2851,Saturn 3 (1980),Adventure|Sci-Fi|Thriller
3454,3553,Gossip (2000),Drama|Thriller


In [125]:
df_movies_dup.shape

(196, 3)

In [126]:
df_movies_dup[df_movies_dup['title'] == 'Gossip (2000)']

,movieId,title,genres
3454,3553,Gossip (2000),Drama|Thriller
44375,168088,Gossip (2000),Comedy|Drama


In [127]:
df_movies_dup[df_movies_dup['title'] == 'Inside (2012)']

,movieId,title,genres
28373,131556,Inside (2012),(no genres listed)
34147,144748,Inside (2012),Horror


In [128]:
df_movies_dup[df_movies_dup['title'] == 'Saturn 3 (1980)']

,movieId,title,genres
2759,2851,Saturn 3 (1980),Adventure|Sci-Fi|Thriller
44502,168358,Saturn 3 (1980),Sci-Fi|Thriller


In [129]:
df_movies_dup[df_movies_dup['genres'] == '(no genres listed)']


,movieId,title,genres
28373,131556,Inside (2012),(no genres listed)
31457,138656,Black Field (2009),(no genres listed)
34186,144830,The Tunnel (1933),(no genres listed)
39216,156686,Another World (2014),(no genres listed)
41061,160868,Escalation (1968),(no genres listed)
48048,175857,Family Life (1971),(no genres listed)
49071,177993,Escape Room (2017),(no genres listed)
49221,178403,The Forest (2016),(no genres listed)
49867,179783,Let There Be Light (2017),(no genres listed)
50735,181675,Apparition (2014),(no genres listed)


# Observations:
- 98 movies are duplicate, need to drop list of duplicate movie entries which have ' no genre listed' present
- then remove the rest of the duplicates 
- In most cases the first entry has more genres
- no genres listes of duplicates to be dropped first

In [130]:
# dropping all the duplicates with no genre listes category
df_movies_dup = df_movies_dup[df_movies_dup['genres'] != '(no genres listed)']
df_movies_dup

,movieId,title,genres
580,588,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical
1710,1788,Men with Guns (1997),Action|Drama
2553,2644,Dracula (1931),Horror
2759,2851,Saturn 3 (1980),Adventure|Sci-Fi|Thriller
3454,3553,Gossip (2000),Drama|Thriller
...,...,...,...
61521,206117,The Lonely Island Presents: The Unauthorized B...,Comedy
61525,206125,Lost & Found (2018),Comedy|Drama
61697,206674,Camino (2016),Comedy
61800,206925,The Plague (2006),Documentary


In [131]:
# checking if any no genre present
df_movies_dup[df_movies_dup['genres'] == '(no genres listed)']

,movieId,title,genres


In [132]:
df_movies_dup.drop_duplicates(subset='title', inplace = True, keep= 'first')
df_movies_dup

,movieId,title,genres
580,588,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical
1710,1788,Men with Guns (1997),Action|Drama
2553,2644,Dracula (1931),Horror
2759,2851,Saturn 3 (1980),Adventure|Sci-Fi|Thriller
3454,3553,Gossip (2000),Drama|Thriller
...,...,...,...
57843,197281,Lost & Found (2018),Animation
58728,199219,I See You (2019),Thriller
59537,201074,Beats (2019),Drama
60497,203449,The Lonely Island Presents: The Unauthorized B...,Comedy


In [133]:
# removing all the duplicate entries from the movies
df_movies.drop_duplicates(subset='title', inplace = True, keep= False)
df_movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)


In [134]:
# appending the movies df with the ones duplicated which was cleaned
df_movies.append(df_movies_dup)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
57843,197281,Lost & Found (2018),Animation
58728,199219,I See You (2019),Thriller
59537,201074,Beats (2019),Drama
60497,203449,The Lonely Island Presents: The Unauthorized B...,Comedy


In [135]:
# checking if any duplicates present in the df_movies dataset
duplicate = df_movies[df_movies.duplicated(subset='title')]
duplicate

,movieId,title,genres


In [136]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


### TODO 
- Need to clean the movie title, remove year and make a year column separate
- Need to separate the genres


In [137]:
# Split title and release year in separate columns in movies dataframe. Convert year to timestamp.
# movies['year'] = movies.title.str.extract("\((\d{4})\)", expand=True)
# movies.year = pd.to_datetime(movies.year, format='%Y')
# movies.year = movies.year.dt.year # As there are some NaN years, resulting type will be float (decimals)
# movies.title = movies.title.str[:-7]

df_movies['year'] = df_movies.title.str.extract('\((\d{4})\)', expand=True)
df_movies['year'].head()

0    1995
1    1995
2    1995
3    1995
4    1995
Name: year, dtype: object

In [138]:
df_movies['year'].isnull().sum()

410

In [139]:
df_movies['title'] = df_movies['title'].str[:-7]

In [140]:
df_movies.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [141]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62227 entries, 0 to 62422
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  62227 non-null  int64 
 1   title    62227 non-null  object
 2   genres   62227 non-null  object
 3   year     61817 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.4+ MB


In [142]:
# year is in object format, need to convert to datetime format
df_movies['year'] = pd.to_datetime(df_movies['year'], format='%Y')
df_movies['year'] = df_movies['year'].dt.year

https://stackoverflow.com/questions/42789324/pandas-fillna-mode

In [143]:
df_movies['year'].isna().sum()

410

In [144]:
df_movies['year'].mode()

0    2015.0
dtype: float64

In [145]:
df_movies['year'] = df_movies['year'].fillna(df_movies['year'].mode()[0])

In [146]:
df_movies['year'].isnull().sum()

0

In [148]:
## Fix the genres column
df_movies['genres']=df_movies.genres.apply(lambda x:x.split('|'))
df_movies.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995.0
4,5,Father of the Bride Part II,[Comedy],1995.0


In [150]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62227 entries, 0 to 62422
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  62227 non-null  int64  
 1   title    62227 non-null  object 
 2   genres   62227 non-null  object 
 3   year     62227 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 2.4+ MB


### References Used:
- https://stackoverflow.com/questions/14946494/how-to-subtract-one-dataframe-from-another
- https://stackoverflow.com/questions/23284409/how-to-subtract-rows-of-one-pandas-data-frame-from-another/26678916
- https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html
- https://www.geeksforgeeks.org/find-duplicate-rows-in-a-dataframe-based-on-all-or-selected-columns/
- https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html


### rating.csv that contains ratings of movies by users:

In [ ]:
# importing the ratings dataset
df_ratings = pd.read_csv('/content/drive/MyDrive/ML_Datasets/movielens_dataset/ratings.csv')
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [ ]:
# shape of the ratings dataset
df_ratings.shape

(25000095, 4)

In [ ]:
# columns in the ratings dataset
df_ratings.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

In [ ]:
df_ratings.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [ ]:
df_ratings['rating'].value_counts()

4.0    6639798
3.0    4896928
5.0    3612474
3.5    3177318
4.5    2200539
2.0    1640868
2.5    1262797
1.0     776815
1.5     399490
0.5     393068
Name: rating, dtype: int64

### Observations
- No NULL data
- rating goes from 0.5 to 5.0

## References Used:
- https://www.analyticsvidhya.com/blog/2020/11/create-your-own-movie-movie-recommendation-system/
